# Data Loader

In [1]:
from model.CANnoloAutoencoder import CANnoloAutoencoder
from data_helpers.CANDataset import CANDataset
from dotenv import load_dotenv

import os

load_dotenv()
data_path = os.getenv('DATA_PATH')
dataset = CANDataset(data_path, log_verbosity=1)


Found ambient and attack directories.
Loading CAN metadata...
Parquet files found...
Found processed parquet files...
Loading processed parquet files...
Loading processing data into 'CANData' structure


In [2]:
dataset.ambient_data._keys

['ambient_dyno_drive_basic_long',
 'ambient_dyno_drive_basic_short',
 'ambient_dyno_drive_benign_anomaly',
 'ambient_dyno_drive_extended_long',
 'ambient_dyno_drive_extended_short',
 'ambient_dyno_drive_radio_infotainment',
 'ambient_dyno_drive_winter',
 'ambient_dyno_exercise_all_bits',
 'ambient_dyno_idle_radio_infotainment',
 'ambient_dyno_reverse',
 'ambient_highway_street_driving_diagnostics',
 'ambient_highway_street_driving_long']

# Example Dataframe and Preprocessing Discussion

In [3]:
# dataset.ambient_data.ambient_dyno_drive_basic_short.head()
dataset.ambient_data.ambient_dyno_drive_basic_short.tail()

,time,aid,data,filename,delta_time_last_msg,delta_time_last_same_aid,actual_attack
996475,444.749609,1072,223C87A2248014A0,ambient_dyno_drive_basic_short.log,1.072884e-06,0.100046,False
996476,444.749610,1760,0000000000000000,ambient_dyno_drive_basic_short.log,9.536743e-07,0.008092,False
996477,444.749611,1255,00000013038C0000,ambient_dyno_drive_basic_short.log,1.072884e-06,0.100046,False
996478,444.750605,339,00000000000C0002,ambient_dyno_drive_basic_short.log,9.939671e-04,0.020714,False
996479,444.750607,661,0000000000000040,ambient_dyno_drive_basic_short.log,2.026558e-06,0.031103,False


In [4]:
unique_can_ids = dataset.get_unique_can_ids()
num_can_ids = len(unique_can_ids)

print(f'Number of unique CAN IDs: {num_can_ids}')
print(f'Unique CAN IDs: {unique_can_ids}')

Number of unique CAN IDs: 105
Unique CAN IDs: [ 813 1694  293  737  852 1505 1760  354  167 1255  339  661 1634 1668
 1590   14  727  412  208  263   51  628  192  996  961 1628 1399  403
  526 1314  651  870 1408 1076 1031 1176  560 1459  186   61 1124 1277
 1372   60  519 1225  470  244  458  675  705 1788  622  569  215 1331
 1175  692 1413  738  837 1072  304 1644  722 1227  640  117   58  953
  541 1262 1533 1693  253  627  452  676  881 1661 1307  420  204  683
 1751  300 1512 1560  485  778  241   65  663 1455   37 1621  426 1049
    6 1398  695  248  631  930 1649]


# Config file for custom format for features

In [5]:

config = {
    "batch_size": 128,
    # "delta_time_last_msg": {
    #     "specific_to_can_id": False,
    #     "records_back": 30
    # },
    "delta_time_last_same_aid": {
        "specific_to_can_id": True,
        "records_back": 40
    },
}

ambient_loader, validation_loader, attack_loader = dataset.get_dataloaders(config)

In [6]:
from helpers import Timing

with Timing():
    ids, features, label = ambient_loader.__getitem__(0)
    print(f'ids: {ids}')
    print(f'features: {features}')
    print(f'label: {label}')
    ambient_loader.reset()

ids: tensor([ 813,  953,  541,  263,  241,  263,  186,   65,  263,  663, 1455,  813,
          60,  519, 1225,  470, 1049,  263,  186,  263,  813,  263,  186,  263,
         263,  186, 1307,  813,   60,  519, 1225,  470,  263,  426,  263,  186,
         778,  452,  676,  813,  263,  263,  186,  263,  813,   60,  519, 1225,
         470,  263,  186,    6,  263,  813,  263,  186,  263,  263,  186, 1307,
         813,   60,  519, 1225,  470,  263, 1398,  263,  186,  813,  263, 1331,
         263,  186,  263,  452,  676,  813,   60,  519, 1225,  470,  263,  186,
        1049,  263,  813,  263,  186,  241,  263,  263,  186,  663, 1307,  813,
          60,  519, 1225,  470,  263,  248,  263,  186,  813,  263,  263,  186,
         263,  813,   60,  519, 1225,  470,  263,  186,  263,  452,  676,  813,
         263,  186,  263,  263,  186, 1307,  813,   60])
features: tensor([[0.0510, 0.0490, 0.0531,  ..., 0.0501, 0.0500, 0.0491],
        [0.9998, 1.0002, 0.9994,  ..., 0.9997, 0.9999, 0.9998],


In [7]:
from helpers import calculate_feature_vec_length, Timing

feature_vec_length = calculate_feature_vec_length(config)
print(f'Feature vector length: {feature_vec_length}')

Feature vector length: 40


# Training

In [8]:
model_config = {
    "embedding_dim": num_can_ids,
    "lstm_units": 128,
    "dense_units": 256,
    "dropout_rate": 0.2,
    "num_embeddings": max(unique_can_ids) + 1,
    "feature_vec_length": feature_vec_length
}

model = CANnoloAutoencoder(**model_config, force_cpu=True)

Forcing CPU usage...


In [9]:
model.train_loop(train_loader=ambient_loader, 
                 validation_loader=validation_loader, 
                 training_metadata_file="training_metadata.tsv", 
                 model_save_directory="models", psuedo_epoch_size=50, validation_epoch_size=10)

Psuedo Epoch 1, Validation Loss: 0.00485793836414814
Epoch 0, Average Training Loss: 0.006286328565329314


KeyboardInterrupt: 

# Testing

In [186]:
from model.CANnoloAttackDetector import CANnoloAttackDetector
from helpers import seperate_attack_loader, calculate_metrics

model_path = "../archive/canolo_model_112.pt"
threshold = 2.8e-06

detector = CANnoloAttackDetector(model_path, threshold, model_config)

MPS is available. Using MPS...


In [187]:
attack_loaders = seperate_attack_loader(attack_loader, config, remove_non_labelled=True)

No attack labels in accelerator_attack_drive_1.log
No attack labels in accelerator_attack_drive_2.log
No attack labels in accelerator_attack_reverse_1.log
No attack labels in accelerator_attack_reverse_2.log
Found attack labels in correlated_signal_attack_1.log
Found attack labels in correlated_signal_attack_2.log
Found attack labels in correlated_signal_attack_3.log
Found attack labels in fuzzing_attack_1.log
Found attack labels in fuzzing_attack_2.log
Found attack labels in fuzzing_attack_3.log
Found attack labels in max_engine_coolant_temp_attack.log
Found attack labels in max_speedometer_attack_1.log
Found attack labels in max_speedometer_attack_2.log
Found attack labels in max_speedometer_attack_3.log
Found attack labels in reverse_light_off_attack_1.log
Found attack labels in reverse_light_off_attack_2.log
Found attack labels in reverse_light_off_attack_3.log
Found attack labels in reverse_light_on_attack_1.log
Found attack labels in reverse_light_on_attack_2.log
Found attack lab

In [188]:
accuracy, precision, recall, f1_score, confusion_matrix = calculate_metrics(detector.detect_attacks(attack_loaders[5]))

In [189]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 score: {f1_score}')
print(f'Confusion matrix: {confusion_matrix}')

Accuracy: 0.8780244449987528
Precision: 0.0013623978201634877
Recall: 0.6666666666666666
F1 score: 0.0027192386131883075
Confusion matrix: [[2, 1466], [1, 10558]]
